# Using Data in GMPL Magic

This example is based on the American Engine Co. example in "Deterministic Operations Research: Models and Methods" by David J. Rader (Wiley, 2010).

> American Engine Co. produces two engines, one for trucks and one for cars.
During the next 3 months, they anticipate the following demands for their engines:

<table>
    <tr>
        <td></td>
        <td>Month 1</td>
        <td>Month 2</td>
        <td>Month 3</td>
    </tr>
    <tr>
        <td>Truck engines</td>
        <td>400</td>
        <td>300</td>
        <td>500</td>
    </tr>
    <tr>
        <td>Car engines</td>
        <td>800</td>
        <td>500</td>
        <td>600</td>
    </tr>
</table>

> Each month's demand must be fully satisfied.
During each month, at most 1000 engines (combined) can be produced.
Each truck engine requires 10 hours of labor to produce and costs \$2000 in supplies,
while each car engine requires 8 hours of labor and costs \$1500 in supplies.
At most 9000 hours are available each month.
At the beginning of month 1, 100 truck engines and 200 car engines are inventory.
At the end of each month, a holding cost of \$150 per engine is assigned to any engine in inventory.
At the end of the third month, management wants to have at least 100 of each engine in inventory.
How can we meet monthly demand at a minimum cost?

This problem can be solved with the following linear program:

**Sets:**
$$
\begin{aligned}
E & = \text{set of engine types}\\
T & = \text{set of time periods (months)}
\end{aligned}
$$

**Parameters:**
$$
\begin{alignedat}{2}
    d_{e,t} & = \text{demand for type $e$ engines in month $t$} &\quad& \text{for } e \in E, t \in T\\
    a_{e} & = \text{labor required to produce one type $e$ engine} &\quad& \text{for } e \in E\\
    c_{e} & = \text{cost of producing one type $e$ engine} &\quad& \text{for } e \in E\\
    I_{e,0} & = \text{initial inventory of type $e$ engines} &\quad& \text{for } e \in E
\end{alignedat}
$$

**Decision variables:**
$$
\begin{alignedat}{2}
    x_{e, t} & = \text{number of type $e$ engines to produce in month $t$} &\quad& \text{for } e \in E, t \in T\\
    y_{e, t} & = \text{number of type $e$ engines in inventory at the end of month $t$} &\quad& \text{for } e \in E, t \in T \cup \{0\}
\end{alignedat}
$$

**Objective function and constraints:**
$$
\begin{alignedat}{2}
\text{minimize} \quad & 150 \sum_{e \in E} \sum_{t \in T} y_{e,t} + \sum_{e \in E} c_e \sum_{t \in T} x_{e,t}\\
\text{subject to} \quad & \sum_{e \in E} x_{e,t} \le 1000 &\quad& \text{for } t \in T\\
& \sum_{e \in E} a_{e} x_{e,t} \le 9000 &\quad& \text{for } t \in T\\
& y_{e,t-1} + x_{e,t} = d_{e,t} + y_{e,t} &\quad& \text{for } e \in E, t \in T\\
& y_{e,0} = I_{e,0} &\quad& \text{for } e \in E\\
& y_{e,3} \ge 100 &\quad& \text{for } e \in E
\end{alignedat}
$$

## Before we get started...

First, we need to load GMPL Magic: 

In [ ]:
%load_ext gmplmagic

## Defining a GMPL model

Below, we create a model called `american` based on the linear program above:

In [ ]:
%%model american

# Sets
set E;    # set of engine types
set T;    # set of time periods (months)

# Parameters
param d{e in E, t in T};    # demand for type e engines in month t
param a{e in E};            # labor required to produce one type e engine
param c{e in E};            # cost of producing one type e engine
param I0{e in E};           # initial inventory of type e engines

# Decision variables
var x{e in E, t in T} >= 0;             # number of type e engines to produce in month t
var y{e in E, t in T union {0}} >= 0;   # number of type e engines in inventory at the end of month t

# Objective function
maximize total_cost: 150 * sum{t in T, e in E} y[e,t] + sum{e in E, t in T} c[e] * x[e,t];

# Constraints
subject to engines_per_month{t in T}:
    sum{e in E} x[e,t] <= 1000;

subject to labor_per_month{t in T}:
    sum{e in E} a[e] * x[e,t] <= 9000;

subject to balance{e in E, t in T}:
    y[e,t-1] + x[e,t] = d[e,t] + y[e,t];

subject to initial_inventory{e in E}:
    y[e,0] = I0[e];
    
subject to ending_inventory{e in E}:
    y[e,3] >= 100;
    
end;

## Defining GMPL data

GMPL Magic has a cell magic __`%%data`__. Below, we create a GMPL data section called `american_data` based on the problem description and linear program above:

In [ ]:
%%data american_data

# Sets
set E := "truck" "car";
set T := 1 2 3;

# Parameters
param d :
        1   2   3 :=
truck 400 300 500
car   800 500 600;

param a :=
    truck   10
    car     8;
    
param c :=
    truck   2000
    car     1500;

param I0 :=
    truck   100
    car     200;

end;

## Solving a GMPL model

Now we can use the line magic __`%solve`__ to put the model and data together. Below, we solve the `american` model with the `american_data` data we created above, and put the results of solving this model in a variable called `solution`.

In [ ]:
%solve --result=solution american american_data

Looking at the variable `solution`, we can see a summary of the results from GLPK.

In [ ]:
solution

## Multiple data sections

We can store multiple data sections and use them in the same notebook! 

To illustrate, suppose we acquired some demand information about months 4 and 5.

Below, we create a data section called `american_data2` that incorporates this new data.

In [ ]:
%%data american_data2

# Sets
set E := "truck" "car";
set T := 1 2 3 4 5;

# Parameters
param d :
        1   2   3   4   5 :=
truck 400 300 500 200 300
car   800 500 600 900 700;

param a :=
    truck   10
    car     8;
    
param c :=
    truck   2000
    car     1500;

param I0 :=
    truck   100
    car     200;
    
end;

We can solve the same `american` model but with this new data section in a similar way:

In [ ]:
%solve --result=solution2 american american_data2

Let's inspect the results of solving the model with this new data:

In [ ]:
solution2

## More helpful utilities

You can display all the stored data sections using __`%listdata`__:

In [ ]:
%listdata

You can also inspect a stored model with __`%showdata`__. For example, to see the `american_data` data section stored in memory:

In [ ]:
%showdata american_data